In [1]:
# Importing Libraries
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from mtcnn.mtcnn import MTCNN

In [2]:
# Loading model
model_res = load_model('face_mask_model.h5')

# Defining image height and width accepted by our model
img_width, img_hight = 224, 224

In [3]:
# Loading MTCNN
detector = MTCNN()

#startt  web cam
cap = cv2.VideoCapture(0) 

img_count_full = 0

#parameters for text
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
# org 
org = (1, 1)
class_lable=' '      
# fontScale 
fontScale = 1 
# Line thickness of 2 px 
thickness = 2 

#sart reading images and prediction
while True:
    img_count_full += 1
    
    #read image from webcam
    response, color_img = cap.read()
    
    #if respose is False then break the loop
    if response == False:
        break    

    # Detect the faces
    faces = detector.detect_faces(color_img)
    
    #take face then predict class mask or not mask then draw rectangle and text then display image
    img_count = 0
    for i in range(0,len(faces)):
        bounding_box = faces[i]['box']
        x = bounding_box[0]
        y = bounding_box[1]
        w = bounding_box[2]
        h = bounding_box[3]
        org = (x-10,y-10)
        img_count +=1 
        color_face = color_img[y:y+h,x:x+w] # color face
        cv2.imwrite('MTCNN_detected_faces/%d%dface.jpg'%(img_count_full,img_count),color_face)
        img = load_img('MTCNN_detected_faces/%d%dface.jpg'%(img_count_full,img_count), target_size=(img_width,img_hight))
        
        img = img_to_array(img)/255
        img = np.expand_dims(img,axis=0)
        pred_prob = model_res.predict(img)
        pred=np.argmax(pred_prob)
            
        if pred==0:
            print("User with mask - predic = ",pred_prob[0][0])
            class_lable = "Mask"
            color = (0, 255, 0)
            cv2.rectangle(color_img, (x, y), (x+w, y+h), (0, 255, 0), 3)
            cv2.putText(color_img, class_lable, org, font,fontScale, color, thickness, cv2.LINE_AA)             
        else:
            print('user not wearing mask - prob = ',pred_prob[0][1])
            class_lable = "No Mask"
            color = (0, 0, 255)
            cv2.rectangle(color_img, (x, y), (x+w, y+h), (0, 0, 255), 3)
            cv2.putText(color_img, class_lable, org, font,fontScale, color, thickness, cv2.LINE_AA)          

    # display image
    cv2.imshow('LIVE face mask detection', color_img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture object
cap.release()
cv2.destroyAllWindows()

user not wearing mask - prob =  0.9974388
user not wearing mask - prob =  0.9981109
user not wearing mask - prob =  0.9966736
user not wearing mask - prob =  0.9983582
